In [13]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector

from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [14]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('../configs/universenet/universenet101_2008d_fp16_4x4_mstrain_480_960_20e_coco.py')

root='../../../dataset/'

epoch = 'epoch_24'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
#cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [1]
cfg.work_dir = './work_dirs/universenet101_2008d_fp16_4x4_mstrain_480_960_20e_coco'

#cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [15]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [16]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

/opt/ml/detection/baseline/UniverseNet/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


load checkpoint from local path: ./work_dirs/universenet101_2008d_fp16_4x4_mstrain_480_960_20e_coco/epoch_24.pth


2022-04-07 04:53:51,286 - root - INFO - DeformConv2dPack backbone.layer2.0.convs.0 is upgraded to version 2.
2022-04-07 04:53:51,288 - root - INFO - DeformConv2dPack backbone.layer2.0.convs.1 is upgraded to version 2.
2022-04-07 04:53:51,291 - root - INFO - DeformConv2dPack backbone.layer2.0.convs.2 is upgraded to version 2.
2022-04-07 04:53:51,295 - root - INFO - DeformConv2dPack backbone.layer2.1.convs.0 is upgraded to version 2.
2022-04-07 04:53:51,296 - root - INFO - DeformConv2dPack backbone.layer2.1.convs.1 is upgraded to version 2.
2022-04-07 04:53:51,298 - root - INFO - DeformConv2dPack backbone.layer2.1.convs.2 is upgraded to version 2.
2022-04-07 04:53:51,303 - root - INFO - DeformConv2dPack backbone.layer2.2.convs.0 is upgraded to version 2.
2022-04-07 04:53:51,304 - root - INFO - DeformConv2dPack backbone.layer2.2.convs.1 is upgraded to version 2.
2022-04-07 04:53:51,305 - root - INFO - DeformConv2dPack backbone.layer2.2.convs.2 is upgraded to version 2.
2022-04-07 04:53:51

In [17]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 4.7 task/s, elapsed: 1036s, ETA:     0s

In [18]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.33038026 234.75864 683.5105 297.59363 749....,test/0000.jpg
1,0 0.37847236 757.94635 668.6219 896.78906 983....,test/0001.jpg
2,0 0.3041132 676.97736 230.0205 768.8658 317.24...,test/0002.jpg
3,0 0.36314705 0.0 0.0 970.3714 1002.4933 0 0.14...,test/0003.jpg
4,0 0.67279387 181.67969 261.55563 869.85126 769...,test/0004.jpg
